In [1]:
import pandas as pd
import numpy as np

# Análisis del dataset

In [4]:
df = pd.read_csv("../youtoxic_english_1000.csv",   sep = ",", encoding ='utf-8')

In [5]:
df.head()

,CommentId,VideoId,Text,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsHomophobic,IsReligiousHate,IsRadicalism
0,Ugg2KwwX0V8-aXgCoAEC,04kJtp6pVXI,If only people would just take a step back and...,False,False,False,False,False,False,False,False,False,False,False,False
1,Ugg2s5AzSPioEXgCoAEC,04kJtp6pVXI,Law enforcement is not trained to shoot to app...,True,True,False,False,False,False,False,False,False,False,False,False
2,Ugg3dWTOxryFfHgCoAEC,04kJtp6pVXI,\nDont you reckon them 'black lives matter' ba...,True,True,False,False,True,False,False,False,False,False,False,False
3,Ugg7Gd006w1MPngCoAEC,04kJtp6pVXI,There are a very large number of people who do...,False,False,False,False,False,False,False,False,False,False,False,False
4,Ugg8FfTbbNF8IngCoAEC,04kJtp6pVXI,"The Arab dude is absolutely right, he should h...",False,False,False,False,False,False,False,False,False,False,False,False


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CommentId        1000 non-null   object
 1   VideoId          1000 non-null   object
 2   Text             1000 non-null   object
 3   IsToxic          1000 non-null   bool  
 4   IsAbusive        1000 non-null   bool  
 5   IsThreat         1000 non-null   bool  
 6   IsProvocative    1000 non-null   bool  
 7   IsObscene        1000 non-null   bool  
 8   IsHatespeech     1000 non-null   bool  
 9   IsRacist         1000 non-null   bool  
 10  IsNationalist    1000 non-null   bool  
 11  IsSexist         1000 non-null   bool  
 12  IsHomophobic     1000 non-null   bool  
 13  IsReligiousHate  1000 non-null   bool  
 14  IsRadicalism     1000 non-null   bool  
dtypes: bool(12), object(3)
memory usage: 35.3+ KB


### 1000 registros de comentarios  .  Tipos de datos : 3 columnas de tipo object(string) y 12 columnas de tipo booleanos .
#### Las columnas de tipo string corresponden a las identificaciones del comentario, del video del comentario y el texto del comentario


In [7]:
# Verifico valores nulos
df.isnull().sum()

CommentId          0
VideoId            0
Text               0
IsToxic            0
IsAbusive          0
IsThreat           0
IsProvocative      0
IsObscene          0
IsHatespeech       0
IsRacist           0
IsNationalist      0
IsSexist           0
IsHomophobic       0
IsReligiousHate    0
IsRadicalism       0
dtype: int64

#### No hay valores nulos en ninguna columna

# Limpieza de  comentarios --> limpieza de valores de la columna Text

In [8]:
# Importa spaCy
import spacy
from sklearn.feature_extraction.text import CountVectorizer
import re
import string

In [9]:
# Crea un objeto nlp vacío para procesar ingles --> pipeline de procesamiento
nlp = spacy.blank("en")

#### Contador de palabras y caracteres de nuestros comentarios

In [10]:
#Contador de palabras y letras que usaré cuantas palabras y caracteres se eliminan con la normalización
def contador(df):
    # nuestro target es Text, indicamos la longitud de palabras y se crea nueva columna 
    total_caracter = df['Text'].str.len().sum()
    total_words =  df['Text'].str.split().str.len().sum()
    return total_caracter,total_words

#### La limpieza se realizará en los siguientes puntos:
<ol>
    <li>Eliminar caracteres especificos con libreria re y usando expresiones regulares</li>
    <li>Eliminar los stop words o palabras vacías : palabras no tienen un significado por sí solas, sino que modifican o acompañan a otras, este grupo suele estar conformado por artículos, pronombres, preposiciones, adverbios e incluso algunos verbos. </li>
</ol>

##### Libreria re: Este módulo proporciona operaciones de coincidencia de expresiones regulares
<ol>
    <li>Metodo sub: Retorna la cadena obtenida reemplazando las ocurrencias no superpuestas del pattern («patrón») 
    por lo pasado en el segundo parametro ,en la string pasada  en el tercer parametro.
    Si el patrón no se encuentra, se retorna string sin cambios. </li>
    <li>Metodo escape: Caracteres de escape especiales en pattern (» patrón»). Esto es útil si quieres hacer coincidir una cadena literal arbitraria que puede tener metacaracteres de expresión regular en ella.</li>
</ol>

##### Libreria string: Operaciones comunes de cadena de caracteres
<ol>
    <li>variable punctuation: devuelve el conjunto de símbolos considerados signos de puntuación </li>
    
</ol>


### Metodos de limpieza

In [11]:
"Elimina caracteres poco significativos a fin de reducir la cantidad de palabras a las mas significativas"
def limpiar_texto(text):
    text_mod = re.sub('.*?¿', '', text)
    re.sub('[0-9]+', '', text_mod)
    text_mod = re.sub('[%s]' % re.escape(string.punctuation), '', text_mod)
    text_mod = re.sub('\w*\d\w*', '', text_mod)
    text_mod = re.sub('\n', '', text_mod)
    return text_mod

In [12]:
def eliminar_stop_words(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_stop]
    return " ".join(token for token in tokens)

In [13]:
def limpieza_data(df): 
    df['Text'] = df['Text'].apply(limpiar_texto)
    df['Text'] = df['Text'].apply(eliminar_stop_words)
    return df
        

#### Cantidad de palabras y caracteres de los comentarios antes de la limpieza

In [14]:
total_char,total_words =contador(df)
print(f"total de caracteres: {total_char}")
print(f"total de palabras: {total_words}")

total de caracteres: 185554
total de palabras: 33776


###### Hay 185.554 caracteres y 33.776 palabras  antes de la limpieza

In [15]:
df = limpieza_data(df)

#### Cantidad de palabras y caracteres de los comentarios despues de la limpieza

In [16]:
total_char,total_words =contador(df)
print(f"total de caracteres: {total_char}")
print(f"total de palabras: {total_words}")

total de caracteres: 104921
total de palabras: 15334


## En total se han eliminado :

<ul>
    <li>Caracteres = 185554 -  104921=<h5> 80.633 caracteres eliminados</h5></li>
    <li>Palabras  = 33776 -  15334=<h5>18442 palabras elminadas</h5> </li>
</ul>

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CommentId        1000 non-null   object
 1   VideoId          1000 non-null   object
 2   Text             1000 non-null   object
 3   IsToxic          1000 non-null   bool  
 4   IsAbusive        1000 non-null   bool  
 5   IsThreat         1000 non-null   bool  
 6   IsProvocative    1000 non-null   bool  
 7   IsObscene        1000 non-null   bool  
 8   IsHatespeech     1000 non-null   bool  
 9   IsRacist         1000 non-null   bool  
 10  IsNationalist    1000 non-null   bool  
 11  IsSexist         1000 non-null   bool  
 12  IsHomophobic     1000 non-null   bool  
 13  IsReligiousHate  1000 non-null   bool  
 14  IsRadicalism     1000 non-null   bool  
dtypes: bool(12), object(3)
memory usage: 35.3+ KB


## Guardar dataset limpio

In [18]:
df.to_csv('cleaning_youtoxic.csv',index=False)